In [ ]:
!pip install optuna
!pip install nltk
!pip install transformers
!pip install protobuf
!pip install tiktoken
!pip install sentencepiece
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
import optuna
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00


In [ ]:
import re
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, DatasetDict

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
!nvidia-smi

Tue Feb 25 11:04:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import re

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Define column names based on dataset description
column_names = ["par_id", "art_id", "keyword", "country_code", "text", "label_x"]

# Load the main dataset, skipping disclaimer
df_main = pd.read_csv("dontpatronizeme_pcl.tsv", sep='\t', skiprows=4, names=column_names, dtype={'par_id': str})

# Load the train and dev splits
df_train_split = pd.read_csv("train_semeval_parids-labels.csv", dtype={'par_id': str})
df_dev_split = pd.read_csv("dev_semeval_parids-labels.csv", dtype={'par_id': str})

# Merge to get training and dev sets
df_train = df_main.merge(df_train_split, on="par_id", how="inner")
df_dev = df_main.merge(df_dev_split, on="par_id", how="inner")

# **Use only label_x and binarize labels**
df_train["label"] = df_train["label_x"].apply(lambda x: 1 if x >= 2 else 0)
df_dev["label"] = df_dev["label_x"].apply(lambda x: 1 if x >= 2 else 0)

# Drop unnecessary columns
df_train = df_train[["par_id", "art_id", "keyword", "country_code", "text", "label"]]
df_dev = df_dev[["par_id", "art_id", "keyword", "country_code", "text", "label"]]



### BOW Baseline

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load data
X = df_train["text"]
y = df_train["label"]

# Split df_train into train (80%) and val (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Use df_dev as test set
X_test = df_dev["text"]
y_test = df_dev["label"]

# Replace NaN values in text columns with an empty string
X_train = X_train.fillna("")
X_val = X_val.fillna("")
X_test = X_test.fillna("")

# Convert text to Bag of Words representation
vectorizer = CountVectorizer(lowercase=True, stop_words="english", max_features=5000)
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)
X_test_bow = vectorizer.transform(X_test)  # Now X_test has no NaN values


# Convert text to Bag of Words representation
vectorizer = CountVectorizer(lowercase=True, stop_words="english", max_features=5000)  # Limit vocab size
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)
X_test_bow = vectorizer.transform(X_test)

# Train a simple Logistic Regression model
model = LogisticRegression()
model.fit(X_train_bow, y_train)

# Validate on val set
y_val_pred = model.predict(X_val_bow)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))

# Test on df_dev
y_test_pred = model.predict(X_test_bow)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))



Validation Accuracy: 0.9098507462686567
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      1516
           1       0.55      0.26      0.36       159

    accuracy                           0.91      1675
   macro avg       0.74      0.62      0.65      1675
weighted avg       0.89      0.91      0.90      1675

Test Accuracy: 0.897803247373448
Test Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      1895
           1       0.42      0.19      0.26       199

    accuracy                           0.90      2094
   macro avg       0.67      0.58      0.60      2094
weighted avg       0.87      0.90      0.88      2094



In [ ]:
import numpy as np

# Get feature names
feature_names = np.array(vectorizer.get_feature_names_out())

# Get top predictive words
top_positive = feature_names[np.argsort(model.coef_)[0][-10:]]  # Words most associated with class 1 (PCL)
top_negative = feature_names[np.argsort(model.coef_)[0][:10]]  # Words most associated with class 0 (non-PCL)

print("Top words associated with PCL:", top_positive)
print("Top words associated with non-PCL:", top_negative)


Top words associated with PCL: ['success' 'desperately' 'trapped' 'underprivileged' 'duty' 'hungry'
 'hope' 'donate' 'dreamers' 'christmas']
Top words associated with non-PCL: ['set' 'grow' 'economic' 'basic' 'friday' 'means' 'calling' 'likely'
 'leaving' 'aims']


In [ ]:
import numpy as np

# Get misclassified examples
df_dev["pred_label"] = y_test_pred  # Add model predictions to df_dev
misclassified = df_dev[df_dev["pred_label"] != df_dev["label"]]

# Display a random misclassified example
example = misclassified.sample(1, random_state=42)
print("Text:", example["text"].values[0])
print("True Label:", example["label"].values[0])
print("Predicted Label:", example["pred_label"].values[0])


Text: Her house now holds the memories and keepsakes of a migrant 's life well lived , selflessly offering her home and love to more than 100 British foster children and making history in the process -- along with her late husband -- - by being the first black couple in the Lambeth Council in South London to be allowed to foster white children .
True Label: 1
Predicted Label: 0
